In [2]:
import pandas as pd
from math import pi
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from wordcloud import WordCloud, STOPWORDS
from scipy.stats.stats import pearsonr
import seaborn as sns
import numpy as np
import matplotlib as mpl
plt.rcParams["patch.force_edgecolor"] = True
plt.style.use('fivethirtyeight')
mpl.rc('patch', edgecolor = 'dimgray', linewidth=1)
%matplotlib inline
init_notebook_mode(connected=True)
pd.options.mode.chained_assignment = None # Warning for chained copies disabled

In [4]:
df = pd.read_csv('en.openfoodfacts.org.products.tsv', low_memory=False, sep='\t')
df.head()

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,0000000003087,http://world-en.openfoodfacts.org/product/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000000004530,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,0000000004559,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,0000000016087,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,0000000016094,http://world-en.openfoodfacts.org/product/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
df.info()
df.describe(include = "all")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 356027 entries, 0 to 356026
Columns: 163 entries, code to water-hardness_100g
dtypes: float64(107), object(56)
memory usage: 442.8+ MB


,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,...,fruits-vegetables-nuts_100g,fruits-vegetables-nuts-estimate_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
count,356001,356001,356024,356024,356017,356027,356027,338515,57714,119285,...,3228.000000,404.000000,182.000000,1383.000000,0.0,278.000000,254856.000000,254856.000000,0.0,0.0
unique,356001,356001,3890,224751,224752,216836,216836,249245,42451,15563,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2000000035338,http://world-en.openfoodfacts.org/product/0690...,usda-ndb-import,1489055829,2017-03-09T16:32:00Z,1439141742,2015-08-09T17:35:42Z,Ice Cream,Pâtes alimentaires au blé dur de qualité supér...,500 g,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,1,169868,20,20,30,30,411,201,5285,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,33.392680,60.360124,15.362637,52.102675,NaN,335.790664,9.166137,8.980656,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32.906834,29.262350,3.692658,19.028361,NaN,423.244817,8.999870,9.151757,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.000000,8.000000,6.000000,NaN,0.000000,-15.000000,-15.000000,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,45.000000,12.000000,33.000000,NaN,82.650000,1.000000,1.000000,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,25.000000,58.000000,15.000000,52.000000,NaN,190.950000,10.000000,9.000000,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,55.000000,93.000000,15.000000,70.000000,NaN,378.700000,16.000000,16.000000,NaN,NaN


In [10]:
df.isnull().sum()

code                                        26
url                                         26
creator                                      3
created_t                                    3
created_datetime                            10
last_modified_t                              0
last_modified_datetime                       0
product_name                             17512
generic_name                            298313
quantity                                236742
packaging                               266067
packaging_tags                          266068
brands                                   29050
brands_tags                              29070
categories                              252728
categories_tags                         252752
categories_en                           252726
origins                                 330977
origins_tags                            331015
manufacturing_places                    314019
manufacturing_places_tags               314026
labels       

#### Data Cleaning

1.1 Empty columnns

In [11]:
#drop columns that do not contain any values
df = df.dropna(axis = 1, how = 'all')
df.shape

(356027, 147)

In the following, I will mainly focus on products that contain some information on its nutriments. This information is contained in the variables with the _100g suffix. Hence, I will only keep the products where at least one of these columns contain some information:

In [12]:
# looking at empty rows
composant = []
for s in df.columns:
    if '_100g' in s: composant.append(s)
df_subset_columns = df[composant]
print('empty _100g raws: {}'.format(df_subset_columns.isnull().all(axis=1).sum()))
#___________________
# and deleting them
df_new = df[df_subset_columns.notnull().any(axis=1)]

empty _100g raws: 55904


1.2 Redundant variables

In [13]:
list_columns = ['categories', 'categories_tags', 'categories_en']
df_new[df_new[list_columns].notnull().any(axis=1)][['product_name']+ list_columns][:20:3]

,product_name,categories,categories_tags,categories_en
176,Salade Cesar,Légumes-feuilles,"en:plant-based-foods-and-beverages,en:plant-ba...","Plant-based foods and beverages,Plant-based fo..."
183,Pain Burger Artisan,Boulange,fr:boulange,fr:Boulange
188,Pâté au poulet,Poulets cuisinés,"en:meats,en:poultries,en:chickens,en:cooked-ch...","Meats,Poultries,Chickens,Cooked chicken"
199,Côtes du Rhône Villages 2014,"Boissons,Boissons alcoolisées,Vins,Vins frança...","en:beverages,en:alcoholic-beverages,en:wines,e...","Beverages,Alcoholic beverages,Wines,French win..."
228,Luxury Christmas Pudding,"Sugary snacks,Desserts,Biscuits and cakes,Cake...","en:sugary-snacks,en:biscuits-and-cakes,en:dess...","Sugary snacks,Biscuits and cakes,Desserts,Cake..."
238,Blle Pet 50CL Coca Cola Cherry,en:beverages,"en:beverages,en:sugared-beverages","Beverages,Sugared beverages"
249,7Up,"Sodas au citron,Boissons sucrées","en:plant-based-foods-and-beverages,en:beverage...","Plant-based foods and beverages,Beverages,Plan..."


This short listing shows that there is some redundancy in the information: The categories_tags and categories_en seem similar in their content except for the en: prefix that has been removed from keywords in the second column.

Hence, in case a column exists with the _en suffix, I decide to delete all the columns without any suffix or ending with the _tags string:

In [17]:
columns_to_remove = []
for s in df_new.columns:
    if "_en" in s: 
        t = s.replace('_en', '')
        u = s.replace('_en', '_tags')
        print("{:<20} 'no suffix' -> {} ; '_tags' suffix -> {}".format(s,
                                t in df_new.columns, u in df_new.columns))
        if t in df_new.columns: columns_to_remove.append(t)
        if u in df_new.columns: columns_to_remove.append(u)
df_new.drop(columns_to_remove, axis = 1, inplace = True)

categories_en        'no suffix' -> False ; '_tags' suffix -> False
labels_en            'no suffix' -> False ; '_tags' suffix -> False
countries_en         'no suffix' -> False ; '_tags' suffix -> False
allergens_en         'no suffix' -> False ; '_tags' suffix -> False
traces_en            'no suffix' -> False ; '_tags' suffix -> False
additives_en         'no suffix' -> False ; '_tags' suffix -> False
states_en            'no suffix' -> False ; '_tags' suffix -> False
main_category_en     'no suffix' -> False ; '_tags' suffix -> False


1.3 Fix names in Countries
 - In this list, a few countries were not correctly translated or do not correspond to a valid database entry. A list of official country names is available on wikipedia.

In [20]:
def count_words(df, colonne = 'categories_en'):
    list_words = set()
    for word in df[colonne].str.split(','):
        if isinstance(word, float): continue
        list_words = set().union(word, list_words)       
    print("Nb of categories in '{}': {}".format(colonne, len(list_words)))
    return list(list_words)
list_countries = count_words(df, 'countries_en')

Nb of categories in 'countries_en': 224


In [22]:
country_replacement = {'Tunisie': 'Tunisia', 'Niederlande': 'Netherland', 
    'fr:Bourgogne-aube-nogent-sur-seine':'France', 'fr:Sverige': 'Sweden', 
    'Vereinigtes-konigreich': 'United Kingdom',  'fr:Suiza':'Switzerland',
    'fr:Kamerun':'Cameroon', 'Other-japon':'Japon', 'fr:Marokko':'Morocco', 
    'ar:Tunisie':'Tunisia', 'fr:Marseille-5':'France', 'Australie':'Australia',
    'fr:Marseille-6':'France', 'fr:Scotland':'United Kingdom', 'Soviet Union':'Russia',
    'fr:Vereinigte-staaten-von-amerika':'United States', 'fr:Neukaledonien':'France',
    'fr:Nederland':'Netherland', 'Mayotte':'France', 'Spanje':'Spain', 'Frankrijk':'France',
    'Suisse':'Switzerland', 'fr:Belgie':'Belgium', 'Other-turquie':'Turkey',
    'fr:Spanien':'Spain', 'Pays-bas':'Netherland', 'fr:Saudi-arabien':'Saudi Arabia',
    'Virgin Islands of the United States':'United States', 'fr:England':'England',
    'Allemagne':'Germany', 'fr:Vereinigtes-konigreich':'United Kingdom', 'Belgique':'Belgium',
    'United-states-of-america':'United States', 'Réunion':'France', 'Martinique':'France',
    'Guadeloupe':'France','French Guiana':'France', 'Czech':'Czech Republic', 'Quebec':'Canada',
    'fr:Quebec':'Canada', 'fr:Deutschland':'Germany', 'Saint Pierre and Miquelon':'France'}

In [23]:
#replace the incorrect entries in the dataframe
for index, countries in df['countries_en'].str.split(',').items():
    if isinstance(countries, float): continue
    country_name = []
    found = False
    for s in countries:
        if s in country_replacement.keys():
            found = True
            country_name.append(country_replacement[s])
        else:
            country_name.append(s)
    if found:
        df.loc[index, 'countries_en'] = ','.join(country_name)    

In [24]:
list_countries = count_words(df, 'countries_en')

Nb of categories in 'countries_en': 206


In [25]:
#count country frequency 
country_count = dict()
for country in list(list_countries):
    country_count[country] = df['countries_en'].str.contains(country).sum()

Plot on a map with Plotly

In [26]:
data = dict(type='choropleth',
locations = list(country_count.keys()),
locationmode = 'country names', z = list(country_count.values()),
text = list(country_count.keys()), colorbar = {'title':'Product nb.'},
colorscale=[[0.00, 'rgb(204,255,229)'], [0.01, 'rgb(51,160,44)'],
            [0.02, 'rgb(102,178,255)'], [0.03, 'rgb(166,206,227)'],
            [0.05, 'rgb(31,120,180)'], [0.10, 'rgb(251,154,153)'],
            [0.20, 'rgb(255,255,0)'], [1, 'rgb(227,26,28)']])
layout = dict(title='Availability of products per country',
geo = dict(showframe = True, projection={'type':'Mercator'}))
choromap = go.Figure(data = [data], layout = layout)
iplot(choromap, validate=False)

1.4 Key words